In [ ]:
# init
import pandas as pd
from tqdm import tqdm
# import nltk
# nltk.download()

Import Data

In [ ]:
import glob
import os

pubmed = []

# import pubmed
docList = glob.glob(os.path.join(os.getcwd(), "Datasets/pubmed/", "*.txt"))

for docPath in docList:
    # get doc file name
    docName = os.path.basename(docPath).split('.')[0]
    
    with open(docPath) as doc:
        # insert [class, docs, feature]
        pubmed.append([docName[:3], docName, doc.read().replace('\n', ' ')])

# print(pubmed)

# make dataframe
dataframe = pd.DataFrame(data=pubmed, columns=['class', 'document', 'feature']) 

# export pubmed raw
dataframe.to_csv('pubmed/raw.csv', index=False)

Dataframe raw

In [ ]:
# read csv
dataRaw = pd.read_csv('pubmed/raw.csv')
# get feature
features = dataRaw.loc[:, 'feature']
dataRaw

Preprocesing

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# cleaning
def cleaning(features):
    result = []
    for feature in features:
        regex = re.sub(r'[^a-zA-Z\s]', '', feature)
        result.append(regex)
    return result

# case folding
def caseFolding(features):
    result = []
    for feature in features:
        lower = feature.lower()
        result.append(lower)
    return result

# tokenization
def tokenization(features):
    result = []
    for feature in features:
        token = word_tokenize(feature)
        result.append(token)
    return result

# stopwords removal
def stopWords(features):
    result = []
    stopWords = set(stopwords.words('english'))
    for token in features:
        cleanedFeature = [feature for feature in token if feature not in stopWords]
        result.append(cleanedFeature)
    return result

In [ ]:
# cleaning
clean = cleaning(features)
# print(clean)

case = caseFolding(clean)
# print(casefolding)

token = tokenization(case)
# print(tokenization)

preprocessed = stopWords(token)
# print(preprocessedFeature)

# export pubmed clean
for i in range(len(preprocessed)):
    dataRaw.loc[i, 'feature'] = ' '.join(preprocessed[i])
dataRaw.to_csv('pubmed/clean.csv', index=False)

In [ ]:
# read csv
dataClean = pd.read_csv('pubmed/clean.csv')
# get feature
features = dataClean.loc[:, 'feature']
dataClean

Feature Forming

In [ ]:
from textblob import TextBlob

# BOAW
dataClean.rename(columns={'feature': 'BOAW'}, inplace=True)

for i in tqdm(range(len(features))):
    # BON
    dataClean.loc[i,'BON'] = ' '.join(TextBlob(features[i]).noun_phrases)
    # BONA
    dataClean.loc[i,'BONA'] = ' '.join([word for (word, tag) in TextBlob(features[i]).tags if tag[:2]=='NN' or tag[:2]=='JJ'])

# print(dataClean)

dataClean.to_csv('pubmed/formed.csv', index=False)

In [ ]:
# read csv
dataFormed = pd.read_csv('pubmed/formed.csv')
# get features
classes = dataFormed.loc[:, 'class']
boaw = dataFormed.loc[:, 'BOAW']
bon = dataFormed.loc[:, 'BON']
bona = dataFormed.loc[:, 'BONA']
dataFormed

Term Weighting

In [41]:
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# TF
def tf(features):
    tfVec = CountVectorizer()
    result = tfVec.fit_transform(features)

    featureName = tfVec.get_feature_names_out()
    featureWeight = result.todense().tolist()
    df = pd.DataFrame(featureWeight, columns=featureName)
    return df

tfx = tf(bona)
print(tfx)

# TF-IDF
def tf_idf(features):
    tfIdfVec = TfidfVectorizer()
    result = tfIdfVec.fit_transform(features)

    featureName = tfIdfVec.get_feature_names_out()
    featureWeight = result.todense().tolist()
    df = pd.DataFrame(featureWeight, columns=featureName)
    return df

# idfx = tf_idf(bona)
# print(idfx)

# TF-IDF-ICF
def icf(features, classes):
  icf = []
  C = []

  # count class
  for i in classes:
    if i not in C:
      C.append(i)

  # count term
#   for feature in features:
#     for i in C:
#       if 
    
  
#   for word, val in idfDict.items():
#       idfDict[word] = math.log(N / float(val))
  
  return icf

def tf_idf_icf(features):
    tfIdfIcf = []
    tfIdf = tf_idf(features)

    for word, val in tfIdf.items():
        tfIdfIcf[word] = val * icf[word]
        break
    return tfIdfIcf

# icfx = tf_idf_icf(features)
# print(icfx)

Feature Selection

Clustering

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline